In [ ]:
import networkx as nx

def k_edge_connectivity_basic(G, k):
    """
    Basic algorithm to test if the graph G is at least k-edge-connected.
    Returns True if yes, otherwise False.
    """
    remaining_edges = set(G.edges())
    all_forests = []

    for i in range(k):
        # Create a subgraph without the edges already used in previous forests
        subG = nx.Graph()
        subG.add_nodes_from(G.nodes())
        subG.add_edges_from(remaining_edges)

        # Find a spanning forest in subG
        forest = nx.minimum_spanning_edges(subG, data=False)
        Fi = set(forest)
        all_forests.append(Fi)

        # Remove edges of the found forest from the remaining edge set
        remaining_edges -= Fi

    # Union of all forests
    union_forest = set().union(*all_forests)
    H = nx.Graph()
    H.add_nodes_from(G.nodes())
    H.add_edges_from(union_forest)

    # If H is not connected or is missing some cut, then it's not k-edge-connected
    return nx.is_k_edge_connected(H, k)


In [ ]:
class Sketch:
    def __init__(self, G):
        self.edges = set(G.edges())

    def subtract(self, other):
        self.edges -= other.edges

    def get_spanning_forest(self):
        tempG = nx.Graph()
        tempG.add_edges_from(self.edges)
        return Sketch.from_edges(nx.minimum_spanning_edges(tempG, data=False))

    @classmethod
    def from_edges(cls, edges):
        s = cls(nx.Graph())  # empty graph
        s.edges = set(edges)
        return s


def k_edge_connectivity_sketch(G, k):
    """
    Sketch-emulated algorithm for testing k-edge-connectivity.
    """
    sketches = [Sketch(G) for _ in range(k)]
    forests = []

    for i in range(k):
        # Subtract previous forests from the current sketch
        for j in range(i):
            sketches[i].subtract(forests[j])

        # Get new forest from the current sketch
        Fi = sketches[i].get_spanning_forest()
        forests.append(Fi)

    # Combine all forests into a single graph and check k-edge-connectivity
    final_graph = nx.Graph()
    final_graph.add_nodes_from(G.nodes())
    for forest in forests:
        final_graph.add_edges_from(forest.edges)

    return nx.is_k_edge_connected(final_graph, k)


In [ ]:
import numpy as np
import networkx as nx
import random
from collections import defaultdict


class GraphSketch:
    def __init__(self, num_nodes, dim=512, seed=None):
        self.n = num_nodes
        self.d = dim
        self.sketch = np.zeros((self.n, self.d))
        self.hashes = {}
        self.rng = np.random.default_rng(seed)

    def _edge_to_vector(self, u, v):
        """
        Returns a random vector representation for an edge (u, v).
        Uses signed random projections.
        """
        if (u, v) not in self.hashes and (v, u) not in self.hashes:
            vec = self.rng.choice([-1, 1], size=self.d)
            self.hashes[(u, v)] = vec
            self.hashes[(v, u)] = -vec
        return self.hashes[(u, v)]

    def add_edge(self, u, v):
        vec = self._edge_to_vector(u, v)
        self.sketch[u] += vec
        self.sketch[v] -= vec  # maintains flow conservation

    def remove_edge(self, u, v):
        vec = self._edge_to_vector(u, v)
        self.sketch[u] -= vec
        self.sketch[v] += vec

    def add_graph(self, G):
        for u, v in G.edges():
            self.add_edge(u, v)

    def subtract(self, other):
        self.sketch -= other.sketch

    def copy(self):
        new = GraphSketch(self.n, self.d)
        new.sketch = self.sketch.copy()
        new.hashes = self.hashes
        return new

    def recover_spanning_forest(self, all_edges):
        """
        Greedy sparse recovery: choose edges that help connect components and have significant sketch signal.
        """
        forest = []
        uf = UnionFind(self.n)
        for u, v in sorted(all_edges, key=lambda e: -self.edge_score(e[0], e[1])):
            if uf.find(u) != uf.find(v):
                forest.append((u, v))
                uf.union(u, v)
        return forest

    def edge_score(self, u, v):
        """
        Computes a rough score of how much the edge (u, v) appears in the sketch.
        Higher means more likely it's still in the sketch.
        """
        vec = self._edge_to_vector(u, v)
        score = np.dot(self.sketch[u], vec) - np.dot(self.sketch[v], vec)
        return abs(score)


class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))

    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x

    def union(self, x, y):
        xr = self.find(x)
        yr = self.find(y)
        if xr != yr:
            self.parent[yr] = xr


def k_edge_connectivity_sketch_real(G, k, dim=512):
    n = G.number_of_nodes()
    all_edges = list(G.edges())
    sketches = [GraphSketch(n, dim, seed=i) for i in range(k)]

    # Step 1: Create k independent sketches of G
    for sketch in sketches:
        sketch.add_graph(G)

    forests = []
    removed_edges = set()

    for i in range(k):
        # Subtract previous forests from sketch[i]
        for j in range(i):
            sketches[i].subtract(forests[j]['sketch'])

        # Recover a forest
        forest_edges = sketches[i].recover_spanning_forest(all_edges)

        # Save the sketch of this forest
        forest_sketch = GraphSketch(n, dim)
        for u, v in forest_edges:
            forest_sketch.add_edge(u, v)

        forests.append({'edges': forest_edges, 'sketch': forest_sketch})
        removed_edges.update(forest_edges)

    # Combine all edges from forests
    combined_graph = nx.Graph()
    combined_graph.add_nodes_from(G.nodes())
    for f in forests:
        combined_graph.add_edges_from(f['edges'])

    return nx.is_k_edge_connected(combined_graph, k)


In [ ]:
# Example usage
G = nx.complete_graph(5)  # A 4-edge-connected graph
print(k_edge_connectivity_basic(G, 3))       # Should return True
print(k_edge_connectivity_sketch(G, 3))      # Should return True (simulated)
# Create a 5-node complete graph (which is 4-edge-connected)
G = nx.complete_graph(5)
print("k=3 connected:", k_edge_connectivity_sketch_real(G, 3))
print("k=4 connected:", k_edge_connectivity_sketch_real(G, 4))
print("k=5 connected:", k_edge_connectivity_sketch_real(G, 5))  # Should be False



True
True
k=3 connected: False
k=4 connected: False
k=5 connected: False


In [ ]:
pip install -U memory_profiler

In [ ]:
import time
import psutil
import pandas as pd
import networkx as nx
from memory_profiler import memory_usage


def measure_algorithm(algorithm_fn, G, k):
    """
    Measure execution time and memory usage of an algorithm.
    """
    start_time = time.time()
    mem_usage = memory_usage((algorithm_fn, (G, k)), max_usage=True)
    end_time = time.time()

    result = algorithm_fn(G, k)
    return {
        'result': result,
        'time': end_time - start_time,
        'memory': mem_usage
    }

def generate_or_load_graph(dataset_name='karate'):
    """
    Load a graph dataset or generate one.
    """
    if dataset_name == 'karate':
        return nx.karate_club_graph()
    elif dataset_name == 'erdos_renyi':
        return nx.erdos_renyi_graph(n=100, p=0.1)
    elif dataset_name == 'complete':
        return nx.complete_graph(20)
    elif dataset_name == 'grid':
        return nx.grid_2d_graph(10, 10)
    else:
        raise ValueError("Unknown dataset")

def benchmark_algorithms(graph_name, algorithms, k_values):
    """
    Runs each algorithm on a dataset graph for multiple k-values.
    """
    G = generate_or_load_graph(graph_name)
    results = []

    for k in k_values:
        for name, algo in algorithms.items():
            print(f"Running {name} on {graph_name} (k={k})...")
            metrics = measure_algorithm(algo, G, k)
            results.append({
                'graph': graph_name,
                'k': k,
                'algorithm': name,
                'result': metrics['result'],
                'time (s)': round(metrics['time'], 4),
                'memory (MB)': round(metrics['memory'], 4)
            })

    return pd.DataFrame(results)


In [ ]:
algorithms = {
    'Basic': k_edge_connectivity_basic,
    'Sketch': k_edge_connectivity_sketch_real,
    'Basic_Sketch': k_edge_connectivity_sketch
}

df = benchmark_algorithms('karate', algorithms, k_values=[1, 2, 3])
print(df)

Running Basic on karate (k=1)...
Running Sketch on karate (k=1)...
Running Basic_Sketch on karate (k=1)...
Running Basic on karate (k=2)...
Running Sketch on karate (k=2)...
Running Basic_Sketch on karate (k=2)...
Running Basic on karate (k=3)...
Running Sketch on karate (k=3)...
Running Basic_Sketch on karate (k=3)...
    graph  k     algorithm  result  time (s)  memory (MB)
0  karate  1         Basic    True    0.1259     179.1836
1  karate  1        Sketch    True    0.0972     179.9805
2  karate  1  Basic_Sketch    True    0.1170     179.9805
3  karate  2         Basic   False    0.1233     179.9805
4  karate  2        Sketch   False    0.1111     181.1328
5  karate  2  Basic_Sketch   False    0.0743     179.3242
6  karate  3         Basic   False    0.1347     179.3242
7  karate  3        Sketch   False    0.0898     181.7930
8  karate  3  Basic_Sketch   False    0.0608     179.3320
